### This script examine for different days' recording, if the neurons are the same or not

#### Especially test Kanga, since her probe is fixed

In [ ]:
import pandas as pd
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn
import scipy
import scipy.stats as st
import scipy.io
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
import string
import warnings
import pickle
import json

import os
import glob
import random
from time import time

### function - get body part location for each pair of cameras

In [ ]:
from ana_functions.body_part_locs_eachpair import body_part_locs_eachpair

### function - find social gaze time point

In [ ]:
from ana_functions.find_socialgaze_timepoint import find_socialgaze_timepoint
from ana_functions.find_socialgaze_timepoint_Anipose import find_socialgaze_timepoint_Anipose
from ana_functions.find_socialgaze_timepoint_Anipose_2 import find_socialgaze_timepoint_Anipose_2

### function - define time point of behavioral events

In [ ]:
from ana_functions.bhv_events_timepoint import bhv_events_timepoint
from ana_functions.bhv_events_timepoint_Anipose import bhv_events_timepoint_Anipose

### function - plot behavioral events

In [ ]:
from ana_functions.plot_bhv_events import plot_bhv_events
from ana_functions.plot_bhv_events_levertube import plot_bhv_events_levertube
from ana_functions.tracking_video_Anipose_events_demo import tracking_video_Anipose_events_demo
from ana_functions.plot_continuous_bhv_var import plot_continuous_bhv_var
from ana_functions.draw_self_loop import draw_self_loop
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection

### function - interval between all behavioral events

In [ ]:
from ana_functions.bhv_events_interval import bhv_events_interval

### function - spike analysis

In [ ]:
from ana_functions.spike_analysis_FR_calculation import spike_analysis_FR_calculation

### function - PCA projection

In [ ]:
from ana_functions.PCA_around_bhv_events import PCA_around_bhv_events
from ana_functions.PCA_around_bhv_events_video import PCA_around_bhv_events_video
from ana_functions.confidence_ellipse import confidence_ellipse

from ana_functions.plot_continuous_bhv_var_and_neuralFR import plot_continuous_bhv_var_and_neuralFR

## Analyze each session

### prepare the basic information

In [ ]:

# re-analyze the video or not
reanalyze_video = 0
redo_anystep = 1

# get the fps of the analyzed video
fps = 30

# get the fs for neural recording
fs_spikes = 20000
fs_lfp = 1000

# do OFC sessions or DLPFC sessions
do_OFC = 0
do_DLPFC = 1
if do_OFC:
    savefile_sufix = '_OFCs'
elif do_DLPFC:
    savefile_sufix = '_DLPFCs'
else:
    savefile_sufix = ''
    
# all the videos (no misaligned ones)
# aligned with the audio
# get the session start time from "videosound_bhv_sync.py/.ipynb"
# currently the session_start_time will be manually typed in. It can be updated after a better method is used


# dodson ginger
if 0:
    if do_DLPFC:
        neural_record_conditions = [
                                     # # '20231204_Dodson_withGinger_SR', 
                                     # '20231204_Dodson_withGinger_MC',
                                    # '20240610_Dodson_MC',
                                    '20240531_Dodson_MC_and_SR',
                                   ]
        dates_list = [
                      # # "20231204_SR","20231204_MC",
                      # '20240610_MC',
                      '20240531',
                     ]
        session_start_times = [ 
                                # # 0.00,  107.50, 
                                # 0.00, 
                                0.00,
                              ] # in second
        kilosortvers = [ 
                         # # 2, 2,
                         # 4,
                         4,
                       ]
    elif do_OFC:
        # pick only five sessions for each conditions
        neural_record_conditions = [
                                     '20231101_Dodson_withGinger_MC',
                                     # '20231107_Dodson_withGinger_MC',
                                     # '20231122_Dodson_withGinger_MC',
                                     # '20231129_Dodson_withGinger_MC',
                                     # '20231101_Dodson_withGinger_SR',
                                   ]
        dates_list = [
                      "20231101_MC",
                      # "20231107_MC",
                      # "20231122_MC",
                      # "20231129_MC",
                      # "20231101_SR",
            
                     ]
        session_start_times = [ 
                                 0.00,   
                                #  0.00,  
                                #  0.00,  
                                #  0.00, 
                              ] # in second
        kilosortvers = [ 2, # 2, 2, 2,
                       ]
    
    animal1_fixedorder = ['dodson']
    animal2_fixedorder = ['ginger']

    animal1_filename = "Dodson"
    animal2_filename = "Ginger"

    
# dannon kanga
if 1:
    if do_DLPFC:
        neural_record_conditions = [
                                     '20240508_Kanga_SR',
                                     '20240509_Kanga_MC',
                                     '20240513_Kanga_MC',
                                     '20240514_Kanga_SR',
                                     '20240523_Kanga_MC',
                                     '20240524_Kanga_SR',
                                     '20240606_Kanga_MC',
                                     '20240613_Kanga_MC_DannonAuto',
                                     '20240614_Kanga_MC_DannonAuto',
                                     '20240617_Kanga_MC_DannonAuto',
                                     '20240618_Kanga_MC_KangaAuto',
                                     '20240619_Kanga_MC_KangaAuto',
                                     '20240620_Kanga_MC_KangaAuto',
                                     '20240621_1_Kanga_NoVis',
                                     '20240624_Kanga_NoVis',
                                     '20240626_Kanga_NoVis',
            
                                     '20240808_Kanga_MC_withGinger',
                                     '20240809_Kanga_MC_withGinger',
                                     '20240812_Kanga_MC_withGinger',
                                     '20240813_Kanga_MC_withKoala',
                                     '20240814_Kanga_MC_withKoala',
                                     '20240815_Kanga_MC_withKoala',
                                     '20240819_Kanga_MC_withVermelho',
                                     '20240821_Kanga_MC_withVermelho',
                                     '20240822_Kanga_MC_withVermelho',
            
                                     '20240918_Kanga_EffortBasedMC',
                                     '20241010_Kanga_EffortBasedMC',
                                   ]
        dates_list = [
                      "20240508",
                      "20240509",
                      "20240513",
                      "20240514",
                      "20240523",
                      "20240524",
                      "20240606",
                      "20240613",
                      "20240614",
                      "20240617",
                      "20240618",
                      "20240619",
                      "20240620",
                      "20240621_1",
                      "20240624",
                      "20240626",
            
                      "20240808",
                      "20240809",
                      "20240812",
                      "20240813",
                      "20240814",
                      "20240815",
                      "20240819",
                      "20240821",
                      "20240822",
            
                      "20240918",
                      "20241010",
                     ]
        session_start_times = [ 
                                 0.00,
                                 36.0,
                                 69.5,
                                 0.00,
                                 62.0,
                                 0.00,
                                 89.0,
                                 0.00,
                                 0.00,
                                 0.00,
                                 165.8,
                                 96.0, 
                                 0.00,
                                 0.00,
                                 0.00,
                                 48.0,
                                
                                 59.2,
                                 49.5,
                                 40.0,
                                 50.0,
                                 0.00,
                                 69.8,
                                 85.0,
                                 212.9,
                                 68.5,
            
                                 43.5,
                                 66.0,
                                 
                              ] # in second
        kilosortvers = [
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
            
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
                         4,
            
                         4,
                         4,
                        ]
    elif do_OFC:
        # pick only five sessions for each conditions
        neural_record_conditions = [
                                     
                                   ]
        dates_list = [
                      
                     ]
        session_start_times = [ 
                                
                              ] # in second
        kilosortvers = [
     
                        ] # 2 or 4
    
    animal1_fixedorder = ['dannon']
    animal2_fixedorder = ['kanga']

    animal1_filename = "Dannon"
    animal2_filename = "Kanga"
    


# a test case
if 0:
    dates_list = ["20230324"]
    session_start_times = [5.50] # in second
    animal1_fixedorder = ['eddie']
    animal2_fixedorder = ['sparkle']
    animal1_filename = "Eddie"
    animal2_filename = "Sparkle"

ndates = np.shape(dates_list)[0]

if np.shape(session_start_times)[0] != np.shape(dates_list)[0]:
    exit()    
    
# define bhv events summarizing variables     
tasktypes_all_dates = np.zeros((ndates,1))
coopthres_all_dates = np.zeros((ndates,1))

succ_rate_all_dates = np.zeros((ndates,1))
interpullintv_all_dates = np.zeros((ndates,1))
trialnum_all_dates = np.zeros((ndates,1))

# switch animals to make animal 1 and 2 consistent
owgaze1_num_all_dates = np.zeros((ndates,1))
owgaze2_num_all_dates = np.zeros((ndates,1))
mtgaze1_num_all_dates = np.zeros((ndates,1))
mtgaze2_num_all_dates = np.zeros((ndates,1))
pull1_num_all_dates = np.zeros((ndates,1))
pull2_num_all_dates = np.zeros((ndates,1))

bhv_intv_all_dates = dict.fromkeys(dates_list, [])
pull_trig_events_all_dates = dict.fromkeys(dates_list, [])
pull_trig_events_succtrials_all_dates = dict.fromkeys(dates_list, [])
pull_trig_events_errtrials_all_dates = dict.fromkeys(dates_list, [])


# where to save the summarizing data
data_saved_folder = '/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/3d_recontruction_analysis_self_and_coop_task_data_saved/'

# neural data folder
neural_data_folder = '/gpfs/radev/pi/nandy/jadi_gibbs_data/Marmoset_neural_recording/'



In [ ]:
fig, axs = plt.subplots(1,ndates)
fig.set_figheight(5*1)
fig.set_figwidth(5*ndates)

nchannel = 64
fig2, axs2 = plt.subplots(nchannel,ndates)
fig2.set_figheight(2*nchannel)
fig2.set_figwidth(2*ndates)
#
for ichan in np.arange(0,nchannel,1):
    for idt in np.arange(0,ndates,1):
        axs2[ichan,idt].set_xticks([])
        axs2[ichan,idt].set_xticklabels([])
        axs2[ichan,idt].set_yticks([])
        axs2[ichan,idt].set_yticklabels([])
        axs2[ichan,idt].spines['top'].set_visible(False)
        axs2[ichan,idt].spines['right'].set_visible(False)
        axs2[ichan,idt].spines['bottom'].set_visible(False)
        axs2[ichan,idt].spines['left'].set_visible(False)
        axs2[ichan,idt].set_xlim([0,61])
        axs2[ichan,idt].set_ylim([-210,430])
        #
        if idt == 0:
            axs2[ichan,idt].set_ylabel('fixed depth ch '+str(ichan))
        if ichan == 0:
            axs2[ichan,idt].set_title(dates_list[idt])

for idate in np.arange(0,ndates,1):

    date_tgt = dates_list[idate]
    session_start_time = session_start_times[idate]
    neural_record_condition = neural_record_conditions[idate]
    kilosortver = kilosortvers[idate]


    # get the neural data            

    # load channel maps
    channel_map_file = '/home/ws523/kilisort_spikesorting/Channel-Maps/Neuronexus_whitematter_2x32.mat'
    # channel_map_file = '/home/ws523/kilisort_spikesorting/Channel-Maps/Neuronexus_whitematter_2x32_kilosort4_new.mat'
    channel_map_data = scipy.io.loadmat(channel_map_file)

    # # load spike sorting results
    print('load spike data for '+neural_record_condition)
    if kilosortver == 2:
        spike_time_file = neural_data_folder+neural_record_condition+'/Kilosort/spike_times.npy'
        spike_time_data = np.load(spike_time_file)
    elif kilosortver == 4:
        spike_time_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/spike_times.npy'
        spike_time_data = np.load(spike_time_file)
    # 
    # down-sample the spike recording resolution to 30Hz
    # spike_time_data = spike_time_data/fs_spikes*fps
    # spike_time_data = np.round(spike_time_data)

    #
    if kilosortver == 2:
        spike_clusters_file = neural_data_folder+neural_record_condition+'/Kilosort/spike_clusters.npy'
        spike_clusters_data = np.load(spike_clusters_file)
        spike_channels_data = np.copy(spike_clusters_data)
    elif kilosortver == 4:
        spike_clusters_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/spike_clusters.npy'
        spike_clusters_data = np.load(spike_clusters_file)
        spike_channels_data = np.copy(spike_clusters_data)
    #
    if kilosortver == 2:
        channel_maps_file = neural_data_folder+neural_record_condition+'/Kilosort/channel_map.npy'
        channel_maps_data = np.load(channel_maps_file)
    elif kilosortver == 4:
        channel_maps_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/channel_map.npy'
        channel_maps_data = np.load(channel_maps_file)
    #
    if kilosortver == 2:
        spike_templates_file = neural_data_folder+neural_record_condition+'/Kilosort/spike_templates.npy'
        spike_templates_data = np.load(spike_templates_file)
    elif kilosortver == 4:
        spike_templates_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/spike_templates.npy'
        # spike_templates_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/spike_detection_templates.npy'           
        spike_templates_data = np.load(spike_templates_file)
    #
    if kilosortver == 2:
        templates_file = neural_data_folder+neural_record_condition+'/Kilosort/templates.npy'
        templates_data = np.load(templates_file)
    elif kilosortver == 4:
        templates_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/templates.npy'
        templates_data = np.load(templates_file)
    templates_data = templates_data[:,:,channel_maps_data]
     #
    if kilosortver == 2:
        amplitudes_file = neural_data_folder+neural_record_condition+'/Kilosort/amplitudes.npy'
        spike_amp_data = np.load(amplitudes_file)
    elif kilosortver == 4:
        amplitudes_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/amplitudes.npy'
        spike_amp_data = np.load(amplitudes_file)    
    #
    if kilosortver == 2:
        clusters_info_file = neural_data_folder+neural_record_condition+'/Kilosort/cluster_info.tsv'
        clusters_info_data = pd.read_csv(clusters_info_file,sep="\t")
    elif kilosortver == 4:
        clusters_info_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/cluster_info.tsv'
        clusters_info_data = pd.read_csv(clusters_info_file,sep="\t")
    #
    if kilosortver == 2:
        channel_pos_file = neural_data_folder+neural_record_condition+'/Kilosort/channel_positions.npy'
        channel_pos_data = np.load(channel_pos_file)
    elif kilosortver == 4:
        channel_pos_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/channel_positions.npy'
        channel_pos_data = np.load(channel_pos_file)
    #
    # only get the spikes that are manually checked
    try:
        good_clusters = clusters_info_data[(clusters_info_data.group=='good')|(clusters_info_data.group=='mua')]['cluster_id'].values
        clusters_info_data = clusters_info_data[np.isin(clusters_info_data.cluster_id,good_clusters)]
    except:
        good_clusters = clusters_info_data[(clusters_info_data.group=='good')|(clusters_info_data.group=='mua')]['id'].values
        clusters_info_data = clusters_info_data[np.isin(clusters_info_data.id,good_clusters)]
    #
    # clusters_info_data = clusters_info_data[~pd.isnull(clusters_info_data.group)]
    clusters_info_data = clusters_info_data[~pd.isnull(clusters_info_data.group)]
    #
    spike_time_data = spike_time_data[np.isin(spike_clusters_data,good_clusters)]
    spike_channels_data = spike_channels_data[np.isin(spike_clusters_data,good_clusters)]
    spike_templates_data = spike_templates_data[np.isin(spike_clusters_data,good_clusters)]
    spike_amp_data = spike_amp_data[np.isin(spike_clusters_data,good_clusters)]
    spike_clusters_data = spike_clusters_data[np.isin(spike_clusters_data,good_clusters)]


    # get the spike channel information
    nclusters = np.shape(clusters_info_data)[0]
    #
    for icluster in np.arange(0,nclusters,1):
        try:
            cluster_id = clusters_info_data['id'].iloc[icluster]
        except:
            cluster_id = clusters_info_data['cluster_id'].iloc[icluster]
        spike_channels_data[np.isin(spike_clusters_data,cluster_id)] = clusters_info_data['ch'].iloc[icluster]   
    # 
    # get the channel to depth information, change 2 shanks to 1 shank 
    try:
        channel_depth=np.hstack([channel_pos_data[channel_pos_data[:,0]==0,1]*2,channel_pos_data[channel_pos_data[:,0]==1,1]*2+1])
        # channel_depth=np.hstack([channel_pos_data[channel_pos_data[:,0]==0,1],channel_pos_data[channel_pos_data[:,0]==31.2,1]])            
        # channel_to_depth = np.vstack([channel_maps_data.T[0],channel_depth])
        channel_to_depth = np.vstack([channel_maps_data.T,channel_depth])
    except:
        channel_depth=np.hstack([channel_pos_data[channel_pos_data[:,0]==0,1],channel_pos_data[channel_pos_data[:,0]==31.2,1]])            
        # channel_to_depth = np.vstack([channel_maps_data.T[0],channel_depth])
        channel_to_depth = np.vstack([channel_maps_data.T,channel_depth])
        channel_to_depth[1] = channel_to_depth[1]/30-64 # make the y axis consistent


    # get the mean AP templates (template * mean amplitude)
    nclusters = np.shape(clusters_info_data)[0]
    template_length = np.shape(templates_data)[1]
    #
    try:
        meanAPshape_all = dict.fromkeys(list(map(str,list(clusters_info_data['id']))),[])            
    except:
        meanAPshape_all = dict.fromkeys(list(map(str,list(clusters_info_data['cluster_id']))),[])
    # 
    # use the depth_history to see if one channel has multiple cluster
    depth_iclu_hist = np.array([])
    #
    for icluster in np.arange(0,nclusters,1):
        try:
            cluster_id = clusters_info_data['id'].iloc[icluster]
            channel_id = clusters_info_data['ch'].iloc[icluster]
            depth_iclu = clusters_info_data['depth'].iloc[icluster]/30 # change to 0 - 63
        except:
            cluster_id = clusters_info_data['cluster_id'].iloc[icluster]
            channel_id = clusters_info_data['ch'].iloc[icluster]
            depth_iclu = clusters_info_data['depth'].iloc[icluster]/30 # change to 0 - 63
        tempID = spike_templates_data[np.isin(spike_clusters_data,cluster_id)][0]
        #
        # APshape_template = templates_data[tempID,:,channel_id]
        #
        # find the chanel with the highest amplitude
        a = (abs(templates_data[tempID,:,:]))
        peak_channel = np.unravel_index(a.argmax(), a.shape)[1]
        APshape_template = templates_data[tempID,:,peak_channel]

        APmean_amp = np.nanmean(spike_amp_data[np.isin(spike_clusters_data,cluster_id)])
        APmean_template = APmean_amp * APshape_template
        # normalized 
        # APmean_template = (APmean_template-np.nanmin(APmean_template))/(np.nanmax(APmean_template)-np.nanmin(APmean_template))
        #
        meanAPshape_all[str(cluster_id)] = APmean_template

        # summarizing plot 
        axs[idate].plot(APmean_template)
        axs[idate].set_title(date_tgt)

        # plot separating channel
        FR_iclu = clusters_info_data['fr'].iloc[icluster]
        #
        axs2[int(depth_iclu),idate].plot(APmean_template)
        #
        if np.isin(depth_iclu,depth_iclu_hist):
            axs2[int(depth_iclu),idate].text(40,220,"FR "+"{:.1f}".format(FR_iclu)+"Hz")
        else:
            axs2[int(depth_iclu),idate].text(40,300,"FR "+"{:.1f}".format(FR_iclu)+"Hz")
        #
        depth_iclu_hist = np.hstack((depth_iclu_hist,depth_iclu))

        
savefigs = 1
if savefigs:
    fig2.savefig('meanAPtemplate_differentChannel_differentDays.pdf')
